In [385]:
import tensorflow as tf
import sklearn 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np
from numpy import unique


In [386]:
#Reading the Dataset file 
Malicious_data = pd.read_csv("C:\Jupyter running\DrDoS_DNS.csv", index_col=0, low_memory=False) 
#Display top 5 rows of dataset
Malicious_data.head()
# Display the last 5 rows of dataset
Malicious_data.tail()

c:\users\goyal\appdata\local\programs\python\python38\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,TrafficNature
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
14657,172.16.0.5-192.168.50.1-962-8456-17,172.16.0.5,962,192.168.50.1,8456,17,08:06.6,1,2,0,...,0.0,0,0,0.0,0.0,0,0,0,1,DrDoS_DNS
10496,172.16.0.5-192.168.50.1-963-9091-17,172.16.0.5,963,192.168.50.1,9091,17,08:06.6,1,2,0,...,0.0,0,0,0.0,0.0,0,0,0,1,DrDoS_DNS
8990,172.16.0.5-192.168.50.1-964-5344-17,172.16.0.5,964,192.168.50.1,5344,17,08:06.6,1,2,0,...,0.0,0,0,0.0,0.0,0,0,0,1,DrDoS_DNS
11307,172.16.0.5-192.168.50.1-965-47866-17,172.16.0.5,965,192.168.50.1,47866,17,08:06.6,1,2,0,...,0.0,0,0,0.0,0.0,0,0,0,1,DrDoS_DNS
9627,172.16.0.5-192.168.50.1-966-54980-17,172.16.0.5,966,192.168.50.1,54980,17,08:06.6,46,2,0,...,0.0,0,0,0.0,0.0,0,0,0,1,DrDoS_DNS


In [387]:
Malicious_features = Malicious_data.copy()
Malicious_labels = Malicious_features.pop('TrafficNature')

In [388]:
# To allow Panda to show upto 100 rows of data
pd.options.display.max_rows=100
# To get np array values in two decimal points
np.set_printoptions(precision=2, suppress= True)

In [389]:
# to get number of unique values
counts=Malicious_features.nunique()
print(counts)

Flow ID                         1031905
 Source IP                          106
 Source Port                       2996
 Destination IP                     116
 Destination Port                 65531
 Protocol                             3
 Timestamp                         6862
 Flow Duration                    12978
 Total Fwd Packets                  154
 Total Backward Packets              79
Total Length of Fwd Packets        1616
 Total Length of Bwd Packets        343
 Fwd Packet Length Max             1398
 Fwd Packet Length Min             1342
 Fwd Packet Length Mean            1723
 Fwd Packet Length Std              444
Bwd Packet Length Max               175
 Bwd Packet Length Min              111
 Bwd Packet Length Mean             403
 Bwd Packet Length Std              308
Flow Bytes/s                      16209
 Flow Packets/s                   13765
 Flow IAT Mean                    13807
 Flow IAT Std                     14584
 Flow IAT Max                      8869


In [390]:
# to delete the columns with only 1 unique value
to_del=[i for i,v in enumerate(counts) if v==1]
print(to_del)
colname = Malicious_features.columns[to_del]
print (colname)
print(Malicious_features.shape)
Malicious_features.drop(colname, axis=1, inplace= True)
print(Malicious_features.shape)

[37, 38, 39, 49, 52, 56, 62, 63, 64, 65, 66, 67]
Index([' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', 'FIN Flag Count',
       ' PSH Flag Count', ' ECE Flag Count', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')
(1048575, 86)
(1048575, 74)


In [391]:
# to find the columns with less variance
for i in range(Malicious_features.shape[1]):
 num = len(unique(np.array(Malicious_features)[:,i]))
 percentage = float(num / Malicious_features.shape[0]* 100)
 print('%d, %d, %.1f%%' % (i,num,percentage))

0, 1031905, 98.4%
1, 106, 0.0%
2, 2996, 0.3%
3, 116, 0.0%
4, 65531, 6.2%
5, 3, 0.0%
6, 6862, 0.7%
7, 12978, 1.2%
8, 154, 0.0%
9, 79, 0.0%
10, 1616, 0.2%
11, 343, 0.0%
12, 1398, 0.1%
13, 1342, 0.1%
14, 1723, 0.2%
15, 444, 0.0%
16, 175, 0.0%
17, 111, 0.0%
18, 403, 0.0%
19, 308, 0.0%
20, 16233, 1.5%
21, 13765, 1.3%
22, 13807, 1.3%
23, 14584, 1.4%
24, 8869, 0.8%
25, 418, 0.0%
26, 12243, 1.2%
27, 12931, 1.2%
28, 13522, 1.3%
29, 8098, 0.8%
30, 339, 0.0%
31, 757, 0.1%
32, 761, 0.1%
33, 743, 0.1%
34, 747, 0.1%
35, 27, 0.0%
36, 2, 0.0%
37, 2612, 0.2%
38, 156, 0.0%
39, 13733, 1.3%
40, 1791, 0.2%
41, 1340, 0.1%
42, 1397, 0.1%
43, 1921, 0.2%
44, 589, 0.1%
45, 589, 0.1%
46, 2, 0.0%
47, 2, 0.0%
48, 2, 0.0%
49, 2, 0.0%
50, 2, 0.0%
51, 9, 0.0%
52, 2075, 0.2%
53, 1723, 0.2%
54, 403, 0.0%
55, 2612, 0.2%
56, 154, 0.0%
57, 1616, 0.2%
58, 79, 0.0%
59, 343, 0.0%
60, 200, 0.0%
61, 162, 0.0%
62, 90, 0.0%
63, 365, 0.0%
64, 310, 0.0%
65, 179, 0.0%
66, 299, 0.0%
67, 285, 0.0%
68, 317, 0.0%
69, 199, 0.0%
70, 315,

In [392]:
# Deleting the columns with less variance
to_del=[i for i,v in enumerate(Malicious_features.nunique()) if float(v)/Malicious_features.shape[0]* 100 < 1]
print(to_del)
colname = Malicious_features.columns[to_del]
print (colname)
print(Malicious_features.shape)
Malicious_features.drop(colname, axis=1, inplace= True)
print(Malicious_features.shape)


[1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 24, 25, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
Index([' Source IP', ' Source Port', ' Destination IP', ' Protocol',
       ' Timestamp', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow IAT Max',
       ' Flow IAT Min', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total',
       ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min',
       'Fwd PSH Flags', ' Fwd Header Length', ' Bwd Header Length',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' 

In [393]:
# To see the columns with null values
null_counts=Malicious_features.isnull().sum()
print(null_counts)

Flow ID               0
 Destination Port     0
 Flow Duration        0
Flow Bytes/s         11
 Flow Packets/s       0
 Flow IAT Mean        0
 Flow IAT Std         0
Fwd IAT Total         0
 Fwd IAT Mean         0
 Fwd IAT Std          0
Fwd Packets/s         0
dtype: int64


In [394]:
#To see the columns names with there data type
Malicious_features.dtypes

Flow ID               object
 Destination Port      int64
 Flow Duration         int64
Flow Bytes/s         float64
 Flow Packets/s      float64
 Flow IAT Mean       float64
 Flow IAT Std        float64
Fwd IAT Total          int64
 Fwd IAT Mean        float64
 Fwd IAT Std         float64
Fwd Packets/s        float64
dtype: object

In [395]:
#To find the columns with datatype as object (string)
String_columns=Malicious_features.columns[Malicious_features.dtypes == 'object']
print(String_columns)

Index(['Flow ID'], dtype='object')


In [396]:
# To delete the columns with datatype object
Malicious_features.drop(String_columns,axis=1, inplace= True)
print(Malicious_features.shape)

(1048575, 10)


In [397]:
#To see the columns with not a number value
NAN_counts=Malicious_features.isna().sum()
print(NAN_counts)

 Destination Port     0
 Flow Duration        0
Flow Bytes/s         11
 Flow Packets/s       0
 Flow IAT Mean        0
 Flow IAT Std         0
Fwd IAT Total         0
 Fwd IAT Mean         0
 Fwd IAT Std          0
Fwd Packets/s         0
dtype: int64


In [398]:
col_name = Malicious_features.columns.to_series()[np.isinf(Malicious_features).any()] 
col_name

Flow Bytes/s          Flow Bytes/s
 Flow Packets/s     Flow Packets/s
dtype: object

In [399]:
Malicious_features.drop(columns=col_name, inplace=True)
print(Malicious_features.shape)

(1048575, 8)


In [400]:
Malicious_labels.head()

Unnamed: 0
425     DrDoS_DNS
430     DrDoS_DNS
1654    DrDoS_DNS
2927    DrDoS_DNS
694     DrDoS_DNS
Name: TrafficNature, dtype: object

In [401]:
Malicious_labels.unique()

array(['DrDoS_DNS', 'BENIGN'], dtype=object)

In [402]:
Malicious_features_train = Malicious_features.sample(frac=0.9, random_state=1)
Malicious_features_test = Malicious_features.drop(Malicious_train.index)
Malicious_labels_train = Malicious_labels.sample(frac=0.9, random_state=1)
Malicious_labels_test = Malicious_labels.drop(Malicious_train.index)
print(Malicious_features_train.shape)
print(Malicious_labels_train.shape)
print(Malicious_features_test.shape)
print(Malicious_labels_test.shape)

(943718, 8)
(943718,)
(1026826, 8)
(1026826,)


In [403]:
#Encoding Labels
Train_labels=Malicious_labels_train.values
Test_labels=Malicious_labels_test.values
le= LabelEncoder()

le.fit(Train_labels)
Malicious_labels_train=le.transform(Train_labels)
le.fit(Test_labels)
Malicious_labels_test=le.transform(Test_labels)


print(Test_labels)
print(Malicious_labels_test)

#print (Malicious_labels['Traffic_Nature'])
#le.fit(Malicious_labels.)
#Malicious_labels['Categorical_label']= le.transform(Malicious_labels.Traffic_Nature)

['DrDoS_DNS' 'DrDoS_DNS' 'DrDoS_DNS' ... 'DrDoS_DNS' 'DrDoS_DNS'
 'DrDoS_DNS']
[1 1 1 ... 1 1 1]


In [404]:


#Malicious_features_train = Malicious_train.copy()
#Malicious_labels_train = Malicious_features_train.pop('TrafficNature')
#Malicious_features_test = Malicious_test.copy()
#Malicious_labels_test = Malicious_features_test.pop('TrafficNature')

In [405]:
#
#Malicious_features = np.array(Malicious_features)
#Malicious_features

In [406]:
#normalize = preprocessing.Normalization()
#normalize.adapt(Malicious_features)

In [407]:
X=Malicious_features_train
y=Malicious_labels_train

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X, y)
LR.predict(Malicious_features_test)
round(LR.score(Malicious_features_test,Malicious_labels_test), 4)

c:\users\goyal\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9984